In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

engine = sqlite3.connect('example.db')
def rsq(quary:str) -> pd.DataFrame:
    return pd.read_sql_query(quary, engine)

In [ ]:
SELECT *
FROM information_schema.tables 
WHERE table_catalog = 'olympics_aqi' 
and table_name = 'annual_aqi';

In [13]:
rsq("""SELECT * 
FROM sqlite_master 
WHERE type ='table' AND name NOT LIKE 'sqlite_%' and name ='annual_aqi'
;""")


,type,name,tbl_name,rootpage,sql
0,table,annual_aqi,annual_aqi,1077,"CREATE TABLE ""annual_aqi"" (\n""ID"" INTEGER,\n ..."


In [16]:
rsq("""EXPLAIN
SELECT *
FROM annual_aqi
limit 10;""")

,addr,opcode,p1,p2,p3,p4,p5,comment
0,0,Init,0,23,0,None,0,None
1,1,Integer,10,1,0,None,0,None
2,2,OpenRead,0,1077,0,15,0,None
3,3,Rewind,0,22,0,None,0,None
4,4,Column,0,0,2,None,0,None
5,5,Column,0,1,3,None,0,None
6,6,Column,0,2,4,None,0,None
7,7,Column,0,3,5,None,0,None
8,8,Column,0,4,6,None,0,None
9,9,Column,0,5,7,None,0,None


In [18]:
rsq("""SELECT county_name
  , aqi
  , category
  , aqi_date
FROM daily_aqi_partitioned
where state_code = 15 
ORDER BY aqi;""")

DatabaseError: Execution failed on sql 'SELECT county_name
  , aqi
  , category
  , aqi_date
FROM annual_aqi
where state_code = 15 
ORDER BY aqi;': no such column: aqi

In [19]:
rsq("""SELECT tablename, indexname
FROM pg_indexes;""")

DatabaseError: Execution failed on sql 'SELECT tablename, indexname
FROM pg_indexes;': no such table: pg_indexes

In [20]:
rsq("""SELECT indexname
FROM pg_indexes 
WHERE tablename = 'daily_aqi'; -- Filter condition""")

DatabaseError: Execution failed on sql 'SELECT indexname
FROM pg_indexes 
WHERE tablename = 'daily_aqi'; -- Filter condition': no such table: pg_indexes

In [ ]:
CREATE INDEX defining_parameter_index 
ON daily_aqi (defining_parameter); -- Define the index creation

SELECT indexname -- Check for the index 
FROM pg_indexes 
WHERE tablename = 'daily_aqi';

In [ ]:
"""SELECT category, COUNT(*) as record_cnt, SUM(no_sites) as aqi_monitoring_site_cnt
FROM daily_aqi
WHERE category <> 'Good' AND state_code = 15 -- Filter to Hawaii and non-Good air quality days
GROUP BY category;"""

In [ ]:
SELECT category
  , COUNT(*) as record_cnt
  , SUM(no_sites) as aqi_monitoring_site_cnt
FROM daily_aqi
WHERE category <> 'Good' 
AND state_code = 15 
AND defining_parameter = 'SO2' -- Adding filter by defining_parameter
GROUP BY  category;

In [ ]:
EXPLAIN
SELECT category
  , COUNT(*) as record_cnt
  , SUM(no_sites) as aqi_monitoring_site_cnt
FROM daily_aqi
WHERE defining_parameter = 'SO2'
AND category <> 'Good'
AND state_code = 15 -- Filter to Hawaii
GROUP BY  category;

In [ ]:
"""CREATE INDEX defining_parameter_index ON daily_aqi (defining_parameter);


EXPLAIN
SELECT category
  , COUNT(*) as record_cnt
  , SUM(no_sites) as aqi_monitoring_site_cnt
FROM daily_aqi
WHERE defining_parameter = 'SO2'
AND category <> 'Good'
AND state_code = 15 -- Hawaii
GROUP BY  category;"""